In [ ]:
# Install required packages
#!pip install keras==2.3.1
!pip install tensorflow
!pip install keras==3.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.9.1
    Uninstalling keras-3.9.1:
      Successfully uninstalled keras-3.9.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import glob
import os

from keras import Input
from keras.applications import VGG19
from keras.callbacks import TensorBoard
from keras.layers import BatchNormalization, Activation, LeakyReLU, Add, Dense, Flatten
from keras.layers import Conv2D, UpSampling2D
from keras.models import Model
from keras.optimizers import Adam
from keras import regularizers
from tensorflow.keras.layers import PReLU
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, PReLU, add, Lambda, Reshape, Permute, Dot, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, LeakyReLU, Conv2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, Flatten, Dense
from tensorflow.keras.models import Model
import tensorflow as tf

import random
from numpy import asarray
from itertools import repeat

import imageio
from imageio import imread
from PIL import Image
from skimage.transform import resize as imresize
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Define paths
TRAIN_PATH = '/content/drive/MyDrive/CPR/chest_xray/train/'
VAL_PATH = '/content/drive/MyDrive/CPR/chest_xray/val/'
TEST_PATH = '/content/drive/MyDrive/CPR/chest_xray/test/'
data_path = TRAIN_PATH

# Training parameters
epochs = 1000
batch_size = 16

# Image shapes
low_resolution_shape = (64, 64, 3)
high_resolution_shape = (256, 256, 3)

# Optimizer
common_optimizer = Adam(0.0002, 0.5)

# Set seed for reproducibility
SEED = 2020
tf.random.set_seed(SEED)

In [ ]:
print(os.getcwd())


In [ ]:
def get_train_images(data_path):
    CLASSES = ['NORMAL', 'PNEUMONIA']
    image_list = []

    for class_type in CLASSES:
        image_list.extend(glob.glob(data_path + class_type + '/*'))

    return image_list

In [ ]:

def find_img_dims(image_list):

    min_size = []
    max_size = []

    for i in range(len(image_list)):
        im = Image.open(image_list[i])
        min_size.append(min(im.size))
        max_size.append(max(im.size))

    return min(min_size), max(max_size)

In [ ]:
# # get min/max image sizes

image_list = get_train_images(data_path)
# min_size, max_size = find_img_dims(image_list)
# print('The min and max image dims are {} and {} respectively.'
#       .format(min_size, max_size))

In [ ]:
image_list = image_list[:500]

In [ ]:
len(image_list)

In [ ]:
# image_list ## Cek dataset sebelum dan setelah diproses pengurangan resolusi
# image_list = get_train_images(data_path)

low_resolution_shape = (32, 32, 3)
low_resolution_shape = (64, 64, 3)
high_resolution_shape = (256, 256, 3)

def sample_images(image_list, batch_size, high_resolution_shape, low_resolution_shape):

    """
    Pre-process a batch of training images
    """

    # image_list is the list of all images
    # ransom sample a batch of images
    images_batch = np.random.choice(image_list, size=batch_size)

    lr_images = []
    hr_images = []


    for img in images_batch:

        img1 = imread(img, pilmode='RGB')
        #img1 = imread(img, pilmode='RGB')
        img1 = img1.astype(np.float32)

        # change the size
        img1_high_resolution = imresize(img1, high_resolution_shape)
        img1_low_resolution = imresize(img1, low_resolution_shape)


        # do a random horizontal flip
        if np.random.random() < 0.5:
            img1_high_resolution = np.fliplr(img1_high_resolution)
            img1_low_resolution = np.fliplr(img1_low_resolution)

        hr_images.append(img1_high_resolution)
        lr_images.append(img1_low_resolution)


    # convert lists into numpy ndarrays
    return np.array(hr_images), np.array(lr_images)


hr_images, lr_images = sample_images(image_list,
                                     batch_size=batch_size,
                                     low_resolution_shape=low_resolution_shape,
                                     high_resolution_shape=high_resolution_shape)

# normalize the images
hr_images = hr_images / 127.5 - 1.
lr_images = lr_images / 127.5 - 1.


fig, ax = plt.subplots(1,2, figsize=(10, 6))

for idx in range(len(hr_images)):

    titles = ['HR', 'LR']
    images = [hr_images[idx], lr_images[idx]]

    for i, img in enumerate(images):
        ax[i].imshow((img + 1)/2.0, cmap='gray')
        ax[i].axis("off")
        ax[i].set_title(titles[i])

plt.show()

In [ ]:
def compute_psnr(original_image, generated_image):

    original_image = tf.convert_to_tensor(original_image, dtype=tf.float32)
    generated_image = tf.convert_to_tensor(generated_image, dtype=tf.float32)
    psnr = tf.image.psnr(original_image, generated_image, max_val=1.0)

    return tf.math.reduce_mean(psnr, axis=None, keepdims=False, name=None)

In [ ]:
def plot_psnr(psnr):

    psnr_means = psnr['psnr_quality']
    plt.figure(figsize=(10,8))
    plt.plot(psnr_means)
    plt.xlabel('Epochs')
    plt.ylabel('PSNR')
    plt.title('PSNR')

**SSIM - Structural Similarity Index**

In [ ]:
def compute_ssim(original_image, generated_image):

    original_image = tf.convert_to_tensor(original_image, dtype=tf.float32)
    generated_image = tf.convert_to_tensor(generated_image, dtype=tf.float32)
    ssim = tf.image.ssim(original_image, generated_image, max_val=1.0, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)

    return tf.math.reduce_mean(ssim, axis=None, keepdims=False, name=None)

In [ ]:
def plot_ssim(ssim):

    ssim_means = ssim['ssim_quality']

    plt.figure(figsize=(10,8))
    plt.plot(ssim_means)
    plt.xlabel('Epochs')
    plt.ylabel('SSIM')
    plt.title('SSIM')

**Plot loss function**

In [ ]:
def plot_loss(losses):

    d_loss = losses['d_history']
    g_loss = losses['g_history']

    plt.figure(figsize=(10,8))
    plt.plot(d_loss, label="Discriminator loss")
    plt.plot(g_loss, label="Generator loss")

    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title("Loss")
    plt.legend()

In [ ]:
def sample_images(image_list, batch_size, high_resolution_shape, low_resolution_shape):

    """
    Pre-process a batch of training images
    """

    # image_list is the list of all images
    # ransom sample a batch of images
    images_batch = np.random.choice(image_list, size=batch_size)

    lr_images = []
    hr_images = []


    for img in images_batch:

#         img1 = imread(img, mode=False, pilmode='RGB')
        img1 = imread(img, pilmode='RGB')
        img1 = img1.astype(np.float32)

        # change the size
        img1_high_resolution = imresize(img1, high_resolution_shape)
        img1_low_resolution = imresize(img1, low_resolution_shape)


        # do a random horizontal flip
        if np.random.random() < 0.5:
            img1_high_resolution = np.fliplr(img1_high_resolution)
            img1_low_resolution = np.fliplr(img1_low_resolution)

        hr_images.append(img1_high_resolution)
        lr_images.append(img1_low_resolution)


    # convert lists into numpy ndarrays
    return np.array(hr_images), np.array(lr_images)

In [ ]:
def save_images(original_image, lr_image, sr_image, path):

    """
    Save LR, HR (original) and generated SR
    images in one panel
    """

    fig, ax = plt.subplots(1,3, figsize=(10, 6))

    images = [original_image, lr_image, sr_image]
    titles = ['HR', 'LR','SR - generated']

    for idx,img in enumerate(images):
        # (X + 1)/2 to scale back from [-1,1] to [0,1]
        ax[idx].imshow((img + 1)/2.0, cmap='gray')
        ax[idx].axis("off")
    for idx, title in enumerate(titles):
        ax[idx].set_title('{}'.format(title))

    plt.savefig(path)

SRGAN - VGG19

*Generator*  RUN

In [ ]:
# Define upsampling block with L2 regularization
def upsampling_block(x, filters, weight_decay=1e-5):
    """Upsampling block with L2 regularization"""
    x = Conv2D(filters, kernel_size=3, strides=1, padding='same',
               kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(x)
    x = UpSampling2D(size=2)(x)
    x = LeakyReLU(alpha=0.2)(x)
    return x

def residual_block(x, filters=64, kernel_size=3, strides=1, padding='same', momentum=0.8):
    """
    Define a residual block for the generator network.
    """
    res = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, kernel_regularizer=regularizers.l2(1e-4))(x)
    res = Activation('relu')(res)
    res = BatchNormalization(momentum=momentum)(res)
    res = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, kernel_regularizer=regularizers.l2(1e-4))(res)
    res = BatchNormalization(momentum=momentum)(res)
    res = Add()([res, x])
    return res

def build_enhanced_generator():
    # Define input shape
    input_shape = (64, 64, 3)

    # Input layer
    inputs = Input(shape=input_shape)

    # Pre-residual block
    c1 = Conv2D(64, kernel_size=9, strides=1, padding='same')(inputs)
    c1 = PReLU(shared_axes=[1, 2])(c1)

    # Residual blocks
    r = residual_block(c1, 64, 1)
    for _ in range(15):  # 16 residual blocks in total
        r = residual_block(r, 64, 1)

    # Post-residual block
    c2 = Conv2D(64, kernel_size=3, strides=1, padding='same')(r)
    c2 = BatchNormalization()(c2)
    c2 = add([c1, c2])

    # Add self-attention mechanism
    c2 = self_attention_block(c2, 64)

    # Upsampling blocks
    u1 = upsampling_block(c2, 256)
    u2 = upsampling_block(u1, 256)

    # Output layer
    outputs = Conv2D(3, kernel_size=9, strides=1, padding='same', activation='tanh')(u2)

    # Create model
    model = Model(inputs=inputs, outputs=outputs, name='Generator')
    return model

# Self-attention block as a Keras layer
def self_attention_block(x, channels):
    # Query, key, value transformations
    f = Conv2D(channels // 8, kernel_size=1, strides=1, padding='same')(x)
    g = Conv2D(channels // 8, kernel_size=1, strides=1, padding='same')(x)
    h = Conv2D(channels, kernel_size=1, strides=1, padding='same')(x)

    # Get shape information
    shape = K.int_shape(f)
    height, width = shape[1], shape[2]
    num_channels = shape[3]

    # Reshape using Keras operations with correct dimensions
    f_flat = Reshape((height * width, num_channels))(f)
    g_flat = Reshape((height * width, num_channels))(g)
    h_flat = Reshape((height * width, channels))(h)

    # Transpose g_flat for matrix multiplication
    g_flat_transposed = Permute((2, 1))(g_flat)

    # Calculate attention map
    s = Dot(axes=(2, 1))([f_flat, g_flat_transposed])
    beta = Activation('softmax')(s)

    # Apply attention
    o = Dot(axes=(2, 1))([beta, h_flat])
    o = Reshape((height, width, channels))(o)

    # Scale and add residual connection
    gamma = 0.1  # Fixed scaling factor
    o = Lambda(lambda x: x * gamma)(o)
    x = add([x, o])

    return x

In [ ]:
from tensorflow.keras.layers import Flatten

In [ ]:
# Spectral Normalization for Discriminator
def spectral_norm(w, iterations=1):
    """
    Spectral normalization for weights to improve training stability
    """
    w_shape = w.shape.as_list()
    w = tf.reshape(w, [-1, w_shape[-1]])

    u = tf.Variable(tf.random.normal([1, w_shape[-1]]), trainable=False)

    u_hat = u
    v_hat = None
    for i in range(iterations):
        # Power iteration
        v_ = tf.matmul(u_hat, tf.transpose(w))
        v_hat = tf.nn.l2_normalize(v_)

        u_ = tf.matmul(v_hat, w)
        u_hat = tf.nn.l2_normalize(u_)

    sigma = tf.matmul(tf.matmul(v_hat, w), tf.transpose(u_hat))
    w_norm = w / sigma

    return tf.reshape(w_norm, w_shape)

def build_enhanced_discriminator():
    # define hyperparameters
    leakyrelu_alpha = 0.2
    momentum = 0.7

    # the input is the HR shape
    input_shape = (256, 256, 3)

    # input layer for discriminator
    input_layer = Input(shape=input_shape)

    # 8 convolutional layers with batch normalization and spectral normalization
    dis1 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotUniform())(input_layer)
    dis1 = LeakyReLU(alpha=leakyrelu_alpha)(dis1)

    dis2 = Conv2D(filters=64, kernel_size=3, strides=2, padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotUniform())(dis1)
    dis2 = LeakyReLU(alpha=leakyrelu_alpha)(dis2)
    dis2 = BatchNormalization(momentum=momentum)(dis2)

    dis3 = Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(dis2)
    dis3 = LeakyReLU(alpha=leakyrelu_alpha)(dis3)
    dis3 = BatchNormalization(momentum=momentum)(dis3)

    dis4 = Conv2D(filters=128, kernel_size=3, strides=2, padding='same')(dis3)
    dis4 = LeakyReLU(alpha=leakyrelu_alpha)(dis4)
    dis4 = BatchNormalization(momentum=0.8)(dis4)

    dis5 = Conv2D(256, kernel_size=3, strides=1, padding='same')(dis4)
    dis5 = LeakyReLU(alpha=leakyrelu_alpha)(dis5)
    dis5 = BatchNormalization(momentum=momentum)(dis5)

    dis6 = Conv2D(filters=256, kernel_size=3, strides=2, padding='same')(dis5)
    dis6 = LeakyReLU(alpha=leakyrelu_alpha)(dis6)
    dis6 = BatchNormalization(momentum=momentum)(dis6)

    dis7 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(dis6)
    dis7 = LeakyReLU(alpha=leakyrelu_alpha)(dis7)
    dis7 = BatchNormalization(momentum=momentum)(dis7)

    dis8 = Conv2D(filters=512, kernel_size=3, strides=2, padding='same')(dis7)
    dis8 = LeakyReLU(alpha=leakyrelu_alpha)(dis8)
    dis8 = BatchNormalization(momentum=momentum)(dis8)

     # Add flatten layer before dense layers
    dis8_flat = Flatten()(dis8)

    # fully connected layer
    dis9 = Dense(units=1024)(dis8_flat)
    dis9 = LeakyReLU(alpha=0.2)(dis9)


    # WGAN modification: Remove sigmoid activation for Wasserstein loss
    output = Dense(units=1, kernel_regularizer=regularizers.l2(1e-5))(dis9)

    model = Model(inputs=[input_layer], outputs=[output], name='discriminator')

    return model

In [ ]:
from tensorflow.keras import layers

In [ ]:
def build_VGG19():
    # Load the pre-trained VGG19 model without the top (fully connected) layers
    VGG19_base = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

    # Freeze the layers of the pre-trained model
    for layer in VGG19_base.layers:
        layer.trainable = False

    # Create a new model with the VGG19 base and a custom top
    input_layer = Input(shape=(256, 256, 3))
    features = VGG19_base(input_layer)
    model = Model(inputs=input_layer, outputs=features)

    return model

def build_adversarial_model(generator, discriminator, feature_extractor):
    # input layer for high-resolution images
    input_high_resolution = Input(shape=high_resolution_shape)

    # input layer for low-resolution images
    input_low_resolution = Input(shape=low_resolution_shape)

    # generate high-resolution images from low-resolution images
    generated_high_resolution_images = generator(input_low_resolution)

    # Resize the generated images to match the expected input shape of VGG19
    resized_images = layers.Lambda(lambda x: tf.image.resize(x, (256, 256)))(generated_high_resolution_images)

    # extract feature maps from resized generated images
    features = feature_extractor(resized_images)

    # make the discriminator non-trainable
    discriminator.trainable = False

    # discriminator will give us a probability estimation for the generated high-resolution images
    probs = discriminator(generated_high_resolution_images)

    # create and compile
    adversarial_model = Model([input_low_resolution, input_high_resolution], [probs, features])

    # WGAN modification: Use MSE for both losses since we're not using binary cross-entropy anymore
    adversarial_model.compile(loss=['mse', 'mse'], loss_weights=[1e-3, 1], optimizer=Adam(0.0002, 0.5))

    return adversarial_model

In [ ]:
# Wasserstein loss function 1
def wasserstein_loss(y_true, y_pred):
    return tf.reduce_mean(y_true * y_pred)

# Gradient Penalty Implementation
def gradient_penalty(discriminator, real_samples, fake_samples, batch_size=16):
    """
    Calculates the gradient penalty for WGAN-GP
    """
    # Get random interpolation between real and fake samples
    alpha = tf.random.uniform([batch_size, 1, 1, 1], 0.0, 1.0)
    diff = fake_samples - real_samples
    interpolated = real_samples + alpha * diff

    with tf.GradientTape() as gp_tape:
        gp_tape.watch(interpolated)
        # Get the discriminator output for interpolated images
        pred = discriminator(interpolated, training=True)

    # Calculate gradients with respect to inputs
    grads = gp_tape.gradient(pred, interpolated)
    # Calculate the norm of the gradients
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
    # Calculate the gradient penalty
    gp = tf.reduce_mean((norm - 1.0) ** 2)

    return gp

In [ ]:
# Wasserstein loss function 2
def wasserstein_loss(y_true, y_pred):
    # Reshape y_pred if it has spatial dimensions
    #if len(K.int_shape(y_pred)) > 2:
        #y_pred = Flatten()(y_pred)

    # Ensure y_true has the same shape as y_pred
    #y_true = K.reshape(y_true, K.shape(y_pred))

    return K.mean(y_true * y_pred)

def gradient_penalty(discriminator, real_samples, fake_samples, batch_size=8):
    # Create a custom model for gradient penalty calculation
    real_input = tf.keras.layers.Input(shape=real_samples.shape[1:])
    fake_input = tf.keras.layers.Input(shape=fake_samples.shape[1:])

    # Random weighted average
    alpha = tf.random.uniform(shape=[batch_size, 1, 1, 1], minval=0., maxval=1.)
    interpolated = real_input + alpha * (fake_input - real_input)

    # Get discriminator output for interpolated images
    interpolated_output = discriminator(interpolated)

    # Create a model that takes real and fake samples and outputs gradient penalty
    gp_model = tf.keras.models.Model([real_input, fake_input], interpolated_output)

    # Use gradient tape to calculate gradients
    with tf.GradientTape() as tape:
        tape.watch(interpolated)
        interpolated_output = gp_model([real_samples, fake_samples])

    # Calculate gradients with respect to the interpolated samples
    gradients = tape.gradient(interpolated_output, interpolated)

    # Compute the gradient norm
    gradients_norm = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1, 2, 3]))

    # Return the gradient penalty: (norm - 1)²
    gradient_penalty = tf.reduce_mean(tf.square(gradients_norm - 1.0))
    return gradient_penalty

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import numpy as np
import time
import os
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

In [ ]:
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize as imresize
from skimage.metrics import peak_signal_noise_ratio as psnr

In [ ]:
# Fix the gradient penalty calculation in the train_wgan_gp function
def train_wgan_gp(generator, discriminator, feature_extractor, adversarial_model,
                 data_path, epochs=1000, batch_size=16,
                 low_resolution_shape=(64, 64, 3),
                 high_resolution_shape=(256, 256, 3),
                 lambda_gp=10, n_critic=7,
                 sample_interval=50,
                 checkpoint_interval=50):
    """Train WGAN-GP model"""
    # Create output directory
    output_dir = os.path.join(os.path.dirname(data_path), "outputs")
    os.makedirs(output_dir, exist_ok=True)

    # Get training images
    train_images = get_train_images(data_path)
    print(f"Found {len(train_images)} training images")

    # If no images found, exit
    if len(train_images) == 0:
        print("No training images found!")
        return None, None, None, None, None

    # Initialize optimizers
    d_optimizer = Adam(0.0001, beta_1=0.5, beta_2=0.9)

    # Compile discriminator
    discriminator.compile(
        loss=wasserstein_loss,
        optimizer=d_optimizer
    )

    # Initialize arrays to store metrics
    d_losses = []
    g_losses = []
    psnr_values = []
    ssim_values = []

    # Training loop
    for epoch in range(epochs):
        start_time = time.time()

        # Initialize epoch metrics
        epoch_d_losses = []
        epoch_g_losses = []
        epoch_psnr = []
        epoch_ssim = []

        # Randomly select batch_size images
        idx = np.random.randint(0, len(train_images), batch_size)

        # Load and preprocess images
        hr_images = []
        lr_images = []

        for i in idx:
            img = imread(train_images[i])
            img = img.astype(np.float32)

            # Resize to high and low resolution
            hr_img = imresize(img, high_resolution_shape)
            lr_img = imresize(img, low_resolution_shape)

            # Normalize to [-1, 1]
            hr_img = hr_img / 127.5 - 1.
            lr_img = lr_img / 127.5 - 1.

            hr_images.append(hr_img)
            lr_images.append(lr_img)

        hr_images = np.array(hr_images)
        lr_images = np.array(lr_images)

        # Generate fake images
        generated_high_resolution_images = generator.predict(lr_images)

        # Train discriminator
        for _ in range(n_critic):
            # Labels for Wasserstein loss
            valid = -np.ones((batch_size, 1))  # Want to maximize D(real)
            fake = np.ones((batch_size, 1))    # Want to minimize D(fake)

            # Train on real images
            d_loss_real = discriminator.train_on_batch(hr_images, valid)

            # Train on fake images
            d_loss_fake = discriminator.train_on_batch(generated_high_resolution_images, fake)

            # Calculate gradient penalty
            epsilon = tf.random.uniform([batch_size, 1, 1, 1], 0.0, 1.0)
            interpolated = hr_images + epsilon * (generated_high_resolution_images - hr_images)

            with tf.GradientTape() as gp_tape:
                gp_tape.watch(interpolated)
                # Get the discriminator output for interpolated images
                pred = discriminator(interpolated, training=True)

            # Calculate gradients with respect to inputs
            grads = gp_tape.gradient(pred, interpolated)
            # Calculate the norm of the gradients
            norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
            # Calculate the gradient penalty
            gp = tf.reduce_mean((norm - 1.0) ** 2)

            # Total discriminator loss
            d_loss = d_loss_real - d_loss_fake + lambda_gp * gp
            epoch_d_losses.append(d_loss)

        # Train generator
        image_features = feature_extractor.predict(hr_images)
        g_loss = adversarial_model.train_on_batch([lr_images, hr_images], [valid, image_features])
        epoch_g_losses.append(g_loss[0])

        # Calculate PSNR and SSIM
        for i in range(batch_size):
            # Convert to [0, 1] for metric calculation
            hr = (hr_images[i] + 1) / 2.0
            sr = (generated_high_resolution_images[i] + 1) / 2.0

            # Calculate metrics
            psnr_value = psnr(hr, sr, data_range=1.0)
            ssim_value = ssim(hr, sr, data_range=1.0, win_size=5, channel_axis=2)

            epoch_psnr.append(psnr_value)
            epoch_ssim.append(ssim_value)

        # Append epoch metrics
        d_losses.append(np.mean(epoch_d_losses))
        g_losses.append(np.mean(epoch_g_losses))
        psnr_values.append(np.mean(epoch_psnr))
        ssim_values.append(np.mean(epoch_ssim))

        # Print progress
        elapsed_time = time.time() - start_time
        print(f"Epoch {epoch+1}/{epochs} - {elapsed_time:.2f}s - "
              f"d_loss: {d_losses[-1]:.4f} - g_loss: {g_losses[-1]:.4f} - "
              f"PSNR: {psnr_values[-1]:.4f} - SSIM: {ssim_values[-1]:.4f}")

        # Save sample images at specified intervals
        if (epoch + 1) % sample_interval == 0:
            # Generate and save images
            for index, img in enumerate(generated_high_resolution_images):
                save_images(hr_images[index], lr_images[index], img,
                          path=f"{output_dir}/img_{epoch+1}_{index}")

        # Save model checkpoints
        if (epoch + 1) % checkpoint_interval == 0:  # checkpoint_interval will be defined
            checkpoint_dir = os.path.join(output_dir, "checkpoints")
            os.makedirs(checkpoint_dir, exist_ok=True)

            # Save generator model
            generator.save(os.path.join(checkpoint_dir, f"generator_epoch_{epoch+1}.keras"))

            # Save discriminator model
            discriminator.save(os.path.join(checkpoint_dir, f"discriminator_epoch_{epoch+1}.keras"))

            print(f"Checkpoint saved at epoch {epoch+1}")

    # Compile metrics
    metrics = {
        'd_history': d_losses,
        'g_history': g_losses,
        'psnr_quality': psnr_values,
        'ssim_quality': ssim_values
    }
    # Create plots directory
    plots_dir = os.path.join(output_dir, "plots")
    os.makedirs(plots_dir, exist_ok=True)

    # Plot each metric individually
    for metric_name, values in metrics.items():
        plt.figure(figsize=(10, 6))
        plt.plot(range(len(values)), values, color='#1f77b4', linewidth=1.0)
        plt.title(metric_name.upper(), fontsize=14, fontweight='bold')
        plt.xlabel('Epochs', fontsize=12)
        plt.ylabel(metric_name.upper(), fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(os.path.join(plots_dir, f"{metric_name}.png"), dpi=300)
        plt.close()

    return generator, discriminator, metrics, psnr_values, ssim_values

In [ ]:
# Build models
generator = build_enhanced_generator()
discriminator = build_enhanced_discriminator()
feature_extractor = build_VGG19()

# Compile feature extractor
feature_extractor.trainable = False
feature_extractor.compile(loss='mse', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Build adversarial model
adversarial_model = build_adversarial_model(generator, discriminator, feature_extractor)

# Train the model
generator, discriminator, losses, psnr_values, ssim_values = train_wgan_gp(
    generator, discriminator, feature_extractor, adversarial_model,
    data_path=TRAIN_PATH, epochs=1000, batch_size=16,
    low_resolution_shape=low_resolution_shape,
    high_resolution_shape=high_resolution_shape,
    lambda_gp=10, n_critic=7
)

In [ ]:
def visualize_results(generator, discriminator, metrics, test_images=None, num_samples=5):
    """
    Visualize training results and generate sample images

    Args:
        generator: Trained generator model
        discriminator: Trained discriminator model
        metrics: Dictionary containing training metrics
        test_images: List of test image paths (optional)
        num_samples: Number of samples to generate
    """
    # Create figure for training metrics
    plt.figure(figsize=(20, 10))

    # Plot discriminator and generator losses
    plt.subplot(2, 2, 1)
    plt.plot(metrics['d_history'], label='Discriminator Loss')
    plt.plot(metrics['g_history'], label='Generator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training Losses')

    # Plot PSNR values
    plt.subplot(2, 2, 2)
    plt.plot(metrics['psnr_quality'])
    plt.xlabel('Epoch')
    plt.ylabel('PSNR (dB)')
    plt.title('Peak Signal-to-Noise Ratio')

    # Plot SSIM values
    plt.subplot(2, 2, 3)
    plt.plot(metrics['ssim_quality'])
    plt.xlabel('Epoch')
    plt.ylabel('SSIM')
    plt.title('Structural Similarity Index')

    plt.tight_layout()
    plt.savefig('training_metrics.png')
    plt.show()

    # Generate and visualize sample images if test images are provided
    if test_images is not None and len(test_images) > 0:
        # Select random test images
        indices = np.random.randint(0, len(test_images), num_samples)

        # Create figure for sample images
        plt.figure(figsize=(15, 5 * num_samples))

        for i, idx in enumerate(indices):
            # Load and preprocess image
            img = imread(test_images[idx])
            img = img.astype(np.float32)

            # Create low-resolution version
            lr_img = imresize(img, (64, 64, 3))
            hr_img = imresize(img, (256, 256, 3))

            # Normalize to [-1, 1]
            lr_input = lr_img / 127.5 - 1.
            hr_input = hr_img / 127.5 - 1.

            # Generate super-resolution image
            sr_img = generator.predict(np.expand_dims(lr_input, axis=0))[0]

            # Convert back to [0, 255] for display
            lr_display = (lr_input + 1) * 127.5
            hr_display = (hr_input + 1) * 127.5
            sr_display = (sr_img + 1) * 127.5

            # Clip values to valid range
            lr_display = np.clip(lr_display, 0, 255).astype(np.uint8)
            hr_display = np.clip(hr_display, 0, 255).astype(np.uint8)
            sr_display = np.clip(sr_display, 0, 255).astype(np.uint8)

            # Display images
            plt.subplot(num_samples, 3, i*3 + 1)
            plt.imshow(lr_display)
            plt.title(f'Low Resolution')
            plt.axis('off')

            plt.subplot(num_samples, 3, i*3 + 2)
            plt.imshow(sr_display)
            plt.title(f'Generated (PSNR: {psnr(hr_display/255, sr_display/255, data_range=1.0):.2f}dB)')
            plt.axis('off')

            plt.subplot(num_samples, 3, i*3 + 3)
            plt.imshow(hr_display)
            plt.title('Original High Resolution')
            plt.axis('off')

        plt.tight_layout()
        plt.savefig('sample_results.png')
        plt.show()

# Usage example after training
# visualize_results(generator, discriminator, losses, test_images=get_test_images(TEST_PATH))

In [ ]:
# Get test images
test_images = get_train_images(TEST_PATH)  # or use a separate test set

# Visualize results
visualize_results(generator, discriminator, losses, test_images=test_images)